# 4 Data Preprocessing and Modeling<a id='2_Data_wrangling'></a>

In [1]:
# imports for handling lists
import itertools
# handling warnings
import warnings
# skimage imports
from skimage import data, color, filters, morphology, graph, measure, exposure
from skimage.filters import threshold_otsu, threshold_local, try_all_threshold, sobel, gaussian
from skimage.transform import rotate, rescale, resize
from skimage.feature import canny
from skimage.io import imsave
from skimage.util import img_as_ubyte
# scipy for image
from scipy import ndimage as ndi
# import for file interaction
import os
import io
from pathlib import Path
import cv2
# imports for reading from zip files
import zipfile
from PIL import Image
# array and data frame imports
import numpy as np
import pandas as pd
# helper functions
import helpers as h
# visualization tools
from tqdm.notebook import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

The first step wil be to read the dataset were the labels and image's IDs are stored and check if the latest are unique. In this preprocessing and modeling stage, we will use a different library for interacting with the storage paths, so a couple of more transformations to the tabular data will be necessary, let's explore them.

In [2]:
# loading the dataset saved in 03_EDA
dir_path = r'C:\SPRINGBOARD\retinopathy-detection' # path to repository
labels = pd.read_csv(r'{}\data_processed\labels_sizes_aug.csv'.format(dir_path))

In [3]:
labels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 5 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Image name              1309 non-null   object
 1   Zip File                1309 non-null   object
 2   Image Size              1309 non-null   object
 3   label                   1309 non-null   int64 
 4   Risk of macular edema   1309 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 51.3+ KB


In [4]:
labels.head()

,Image name,Zip File,Image Size,label,Risk of macular edema
0,20051019_38557_0100_PP.tif,Base11.zip,"(1488, 2240, 3)",3,1
1,20051020_43808_0100_PP.tif,Base11.zip,"(1488, 2240, 3)",0,0
2,20051020_43832_0100_PP.tif,Base11.zip,"(1488, 2240, 3)",1,0
3,20051020_43882_0100_PP.tif,Base11.zip,"(1488, 2240, 3)",2,0
4,20051020_43906_0100_PP.tif,Base11.zip,"(1488, 2240, 3)",3,2


The first transformation we will apply to this data frame will be changing the name of the 'Image name' colum for 'image_id'

In [5]:
# renaming image name column
labels.rename(columns={'Image name': 'image_id'}, inplace=True)
labels.head()

,image_id,Zip File,Image Size,label,Risk of macular edema
0,20051019_38557_0100_PP.tif,Base11.zip,"(1488, 2240, 3)",3,1
1,20051020_43808_0100_PP.tif,Base11.zip,"(1488, 2240, 3)",0,0
2,20051020_43832_0100_PP.tif,Base11.zip,"(1488, 2240, 3)",1,0
3,20051020_43882_0100_PP.tif,Base11.zip,"(1488, 2240, 3)",2,0
4,20051020_43906_0100_PP.tif,Base11.zip,"(1488, 2240, 3)",3,2


Now, since we will specify the file extension while handling images, let's just get rid of the '.tif' extension on the image_id values

In [6]:
labels['image_id'] = labels['image_id'].apply(lambda x: x.split('.', 1)[0]) # removing '.tif' string from image_id
labels.head()

,image_id,Zip File,Image Size,label,Risk of macular edema
0,20051019_38557_0100_PP,Base11.zip,"(1488, 2240, 3)",3,1
1,20051020_43808_0100_PP,Base11.zip,"(1488, 2240, 3)",0,0
2,20051020_43832_0100_PP,Base11.zip,"(1488, 2240, 3)",1,0
3,20051020_43882_0100_PP,Base11.zip,"(1488, 2240, 3)",2,0
4,20051020_43906_0100_PP,Base11.zip,"(1488, 2240, 3)",3,2


Good, now it is time to check if all of our images IDs are unique in the data frame (very important for the modeling stage)

In [7]:
# checking the uniqueness of each image ID
labels['image_id'].nunique() == labels.shape[0]

True

Good, there is no need to drop the duplicates in the dataset since all the IDs found are unique. But anyways, let's do it to see no changes in it and for the purpose of how would it be done:

In [8]:
labels.drop_duplicates('image_id', inplace=True)

In [9]:
labels.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 1308
Data columns (total 5 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   image_id                1309 non-null   object
 1   Zip File                1309 non-null   object
 2   Image Size              1309 non-null   object
 3   label                   1309 non-null   int64 
 4   Risk of macular edema   1309 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 61.4+ KB


As expected, no changes made on the dataframe.

Now, since our dataframe is ordered by zip files (since it was read in that way: 'Base11.zip' then 'Base12.zip' and so on), we are going to perform this last transformation in it to break that sorting and shuffle the rows to keep the samples as random as possible.

In [10]:
labels = labels.sample(frac=1)

In [11]:
labels.head()

,image_id,Zip File,Image Size,label,Risk of macular edema
814,20051110_35332_0400_PP,Base31.zip,"(960, 1440, 3)",0,0
98,20051021_51476_0100_PP,Base11.zip,"(1488, 2240, 3)",0,0
487,20060407_39780_0200_PP,Base21.zip,"(960, 1440, 3)",0,0
92,20051021_40074_0100_PP,Base11.zip,"(1488, 2240, 3)",3,0
910,20051201_38280_0400_PP,Base32.zip,"(1536, 2304, 3)",0,0


In [12]:
# saving the data frame to a csv file for later use
#labels.to_csv(r'{}\data_processed\labels_modeling.csv'.format(dir_path), index=False) # saving the dataframe to path

Now, it is time to load and pre process the image data. Our goal here is preprocess the images and the creating a training and validation dataset with folders of images divided by classes, being:

* --> train_folder
  * --> class_0
  * --> class_1
  * --> class_2
  * --> class_3

And then a folder called:

* --> validation_folder
  * --> class_0
  * --> class_1
  * --> class_2
  * --> class_3
  
Let's see how we can achieve this.

In this cell we are going to create our root and save path by using the Path library, this will allow us to handle the paths in an easier way than we were doing in the previous notebooks.

* ROOT_PATH: where we are going to take our image and tabular data files
* SAVE_PATH: where we are going to save our processed files

In [ ]:
ROOT_PATH = Path("C:/SPRINGBOARD/retinopathy-detection/data_processed/data_original/") # take images from
SAVE_PATH = Path("C:/SPRINGBOARD/retinopathy-detection/data_processed/data_model/") # save processed images to

To double check that we in fact have the correct access to the images and integration with the labels at the dataset, let's just plot 9 image examples and show its correcponding label.

In [ ]:
fig, axis = plt.subplots(3, 3, figsize=(9,9))
c = 0
for i in range(3):
    for j in range(3):
        image_id = labels['image_id'].iloc[c]
        tif_path = ROOT_PATH/image_id
        tif_path = tif_path.with_suffix('.tif')
        tif = plt.imread(tif_path)
        
        label = labels['label'].iloc[c]
        
        axis[i][j].imshow(tif, cmap='gray')
        axis[i][j].set_title(label)
        c += 1

Noe it is time to preprocessing and loading our data. In this section we will:

1. Look on our image_id
2. Standarize and resize images
3. Split the train and validation set (80 - 20)
4. Store them in the corresponding directory according to their labels

In [ ]:
sums, sums_squared = 0, 0

# for loop over image_id's to split the train (80%) and test set (20%)
for c, image_id in enumerate(tqdm(labels['image_id'])):
    image_id = labels['image_id'].iloc[c]
    tif_path = ROOT_PATH/image_id
    tif_path = tif_path.with_suffix('.tif')
    tif = plt.imread(tif_path)
    
    # to 'help' the deep learning architecture, we will reshape the images to 244x244
    # this will allow to decrease our computational time for the modeling
    tif_array = cv2.resize(tif, (244, 244)).astype(np.float16) # saving it as float16 to occupy less space in memory
    label = labels['label'].iloc[c]
    
    # deciding in which folder to save the image according to the label
    

## 4.n Summary<a id='2.7_Summary'></a>